In [1]:
import numpy as np
import matplotlib.pyplot as plt
import re

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import warnings

import json
import wandb

# k-meansの収束に関する警告を非表示にする
warnings.filterwarnings('ignore', category=FutureWarning)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [2]:
no_bubble_label = np.load("/home/cygnus/fujimoto/Cygnus-X_CAE/data/zroing_resize_data/resize_data/condition_match/bubble_region_label.npy")
np.sum(no_bubble_label == 0)

5380

In [3]:
api = wandb.Api()
artifact_path = 'yudai22-/condition_matching/training_log:v3'
artifact = api.artifact(artifact_path)

run = artifact.logged_by()


try:
    config_dict = json.loads(run.config)
    latent = config_dict["latent_num"]
    print(latent)
except json.JSONDecodeError:
    # run.config が有効なJSON文字列でなかった場合の処理
    print("エラー: run.config は有効なJSON形式ではありません。")
except TypeError:
    # run.config が文字列でなかった場合の処理 (念のため)
    print("エラー: run.config は文字列型ではありません。")


# latent = run.config["latent_num"]
print(latent)


match = re.findall('/.*/', artifact_path)
log_name = re.search(r"(training.*)", artifact_path).group()

print(log_name)
print(match[0][1:-1])

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

  ········


wandb: Currently logged in as: yudai22 (yudai22-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'value': 100}
{'value': 100}
training_log:v3
condition_matching


In [4]:
all_region_latent = np.load(f"/home/cygnus/fujimoto/Cygnus-X_CAE/analysis/check_latent/latent_list/{match[0][1:-1]}/all/{log_name}_all_latent.npy")
bubble_region_latent = np.load(f"/home/cygnus/fujimoto/Cygnus-X_CAE/analysis/check_latent/latent_list/{match[0][1:-1]}/bubble/{log_name}_bubble_latent.npy")
view_all_region = np.load("/home/cygnus/fujimoto/Cygnus-X_CAE/data/zroing_resize_data/resize_data/condition_match/vflip/CygnusX_cut_truncation_3size_resize_to_mode_percentile_100x100.npy")
bubble_region = np.load("/home/cygnus/fujimoto/Cygnus-X_CAE/data/zroing_resize_data/resize_data/condition_match/vflip/CygnusX_bubble_vflip.npy")

In [7]:
print(len(all_region_latent))

points = np.r_[all_region_latent, bubble_region_latent]
print(len(points))

7925
7956


In [8]:
# --- 2. k-means クラスタリングの実行 ---
# k-meansオブジェクトの初期化と学習
n_clusters = 3

kmeans = KMeans(n_clusters=n_clusters, init='k-means++',random_state=0)
kmeans.fit(points)

# 各データポイントのクラスターラベルを取得
cluster_labels = kmeans.labels_
print(f"k-meansによるクラスターラベルの数: {len(np.unique(cluster_labels))}")

k-meansによるクラスターラベルの数: 3


In [14]:
no_bubble_label_list = list(no_bubble_label)
no_bubble_label_list += [0]*len(bubble_region)

In [16]:
khm_remove = []
for cluster, label in zip(cluster_labels, no_bubble_label_list):
    if label == 1:
        cluster = np.nan
    else:
        pass
    khm_remove.append(cluster)

In [17]:
nan_num = np.sum(np.isnan(khm_remove))
print(len(khm_remove))
print(nan_num)

7956
2545


In [20]:
c_bubble = khm_remove[-31:]
c_all = khm_remove[:-31]
print(len(c_bubble))
print(len(c_all))

31
7925


In [22]:
nan_num = np.sum(np.isnan(khm_remove))
print(len(khm_remove))
print(nan_num)

7956
2545


In [23]:
all_num = len(c_all) - nan_num

for unique_num in np.unique(khm_remove):
    if np.isnan(unique_num):
        continue
    val_all = np.sum(c_all == unique_num)
    val_bubble = np.sum(c_bubble == unique_num)
    
    print(f"| cluster{int(unique_num)} | {'{:.1f}'.format(val_all/all_num*100)}% | {'{:.1f}'.format(val_bubble/len(c_bubble)*100)}% | {val_all}個 | {val_bubble}個 |")

| cluster0 | 22.5% | 32.3% | 1208個 | 10個 |
| cluster1 | 56.8% | 22.6% | 3056個 | 7個 |
| cluster2 | 20.7% | 45.2% | 1116個 | 14個 |


In [7]:
no_bubble_label_list = list(no_bubble_label)
no_bubble_label_list += [0]*len(bubble_region)
len(no_bubble_label_list)

7956

In [10]:
khm_remove = []
for cluster, label in zip(cluster_labels, no_bubble_label_list):
    if label == 1:
        cluster = np.nan
    else:
        pass
    khm_remove.append(cluster)

In [11]:
nan_num = np.sum(np.isnan(khm_remove))
print(len(khm_remove))
print(nan_num)

7956
2545


In [ ]:
# --- 3. t-SNEによる次元削減 (2次元へ) ---
# t-SNEオブジェクトの初期化と変換の実行
# n_components=2 で2次元への削減を指定
# perplexityやlearning_rateはデータの性質に応じて調整が必要な場合があります
tsne = TSNE(n_components=2, perplexity=30, learning_rate='auto', 
            init='random', random_state=random_state, n_jobs=-1)

# 高次元データXを2次元に変換 (t-SNEの実行)
X_tsne = tsne.fit_transform(X)

print(f"t-SNE後のデータ形状: {X_tsne.shape}")

# --- 4. 可視化 (t-SNEの結果をk-meansのクラスターで色付け) ---
plt.figure(figsize=(10, 8))

# 散布図のプロット。c=cluster_labels でクラスターラベルに基づいて色を割り当てる
scatter = plt.scatter(X_tsne[:, 0], X_tsne[:, 1], 
                      c=cluster_labels, 
                      cmap='viridis', # カラーマップの指定
                      s=20, # ポイントのサイズ
                      alpha=0.7) # 透明度

# 凡例の追加
legend1 = plt.legend(*scatter.legend_elements(), 
                     title="Clusters", 
                     loc="lower left", 
                     bbox_to_anchor=(1.05, 0))
plt.gca().add_artist(legend1)

plt.title('t-SNE Visualization Colored by k-means Clusters')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

print("可視化が完了しました。")